# WEGE index
This notebook provides code for the Weighted Endemism including Global Endangerment (WEGE) index as it is described in [Farooq et al. (2020)](https://onlinelibrary.wiley.com/doi/full/10.1111/ddi.13148). WEGE is calculated like this:

$$\text{WEGE} = \sum_{i=1}^\text{N} \sqrt{WE_i}\times ER_i$$

- $WE_i$ weighted endemism for species $i$ (See [Crisp et al. (2002)](https://biology-assets.anu.edu.au/hosted_sites/Crisp/pdfs/Crisp2001_endemism.pdf))
- $ER_i$: probability of extinction of species $i$ (Using the IUCN50 transformation for the ER from [Davis et al. (2008)](https://www.pnas.org/doi/10.1073/pnas.1804906115))

### Weighted Endemism (WE)
[Crisp et al. (2002)](https://biology-assets.anu.edu.au/hosted_sites/Crisp/pdfs/Crisp2001_endemism.pdf)) describes weighted endemism in terms of grid cells, stating:
> ...a single-cell endemic
has the maximum weight of 1, a species occurring in two
cells has a weight of 0.5, and a species occurring in 100 cells
has a weight of 0.01. **To obtain an endemism score for a cell,
these weights are summed for all species occurring in the
cell. We term this measure *weighted endemism*.**

Instead of counting grid cells, we can define the endemism weight for each species as a ratio of areas:

$$WE_i = \frac{\text{"Area where species i is found within the target area"}}{\text{"Target area"}}$$


### Extinction Probability (ER)
[Farooq et al. (2020)](https://onlinelibrary.wiley.com/doi/full/10.1111/ddi.13148) base the extinction probability on the species IUCN Red List categeory, using the transformation from [Davis et al. (2008)](https://www.pnas.org/doi/10.1073/pnas.1804906115). Other transformations exist in the literature, such as [Moors et al. (2008)](https://www.sfu.ca/~amooers/papers/Mooers_etal_PLoSOne08.pdf). Here, I follow [Farooq et al. (2020)](https://onlinelibrary.wiley.com/doi/full/10.1111/ddi.13148) and use the [Davis et al. (2008)](https://www.pnas.org/doi/10.1073/pnas.1804906115) transformation. In addition, also following [Farooq et al. (2020)](https://onlinelibrary.wiley.com/doi/full/10.1111/ddi.13148), I set the extinction probabilty of data deficient (DD) species to the extinction probability of "vulnerable" species following [Bland et al. (2015)](https://conbio.onlinelibrary.wiley.com/doi/10.1111/cobi.12372). This is based on the idea that DD species may be more threatened than some data-sufficient species ([Bland et al., 2015](https://conbio.onlinelibrary.wiley.com/doi/10.1111/cobi.12372), [Borgelt et al., 2022](https://www.nature.com/articles/s42003-022-03638-9))

Here the transformation used for each [IUCN Red List category](https://www.iucnredlist.org/resources/categories-and-criteria):

$$(DD, LC, NT, VU, EN, CR, EW, EX) \mapsto (0.0513, 0.0009, 0.0071, 0.0513, 0.4276, 0.9688, 1.0, 1.0)$$

Where:     
- DD = "Data Deficient"
- LC = "Least Concern"
- NT = "Near Threatened"
- VU = "Vulnerable"
- EN = "Endangered"
- CR = "Critically Endangered"
- EW = "Extinct In The Wild"
- EX = "Extinct"

### Notes
- square root transformation used to improve normality of $WE$ data. Could test whether this transformation is necessary by making historgram of $WE$ and seeing if it is skewed and if square root actually improves normality. 
- which mapping from IUCN category to extinction probability should we use?
- What should the probability of extinction be for "data deficient" species? [Bland et al. (2015)](https://conbio.onlinelibrary.wiley.com/doi/10.1111/cobi.12372) and [Borgelt et al. (2022)](https://www.nature.com/articles/s42003-022-03638-9) suggest DD species may be more threatened than data-sufficient species.

In [1]:
def extinction_risk(cat: str = None) -> float:
    '''calculates extinction risk (ER) for species 
    following Farooq et al. (2020)
    we assign probability of extinction for each IUCN category
    using extinction probabilities 
    from Table S2 in supplemental material of Davis et al (2018).
    Here we use use IUCN50 values, same as Farooq et al. (2020).
    
    Extinction risk for data deficiient (DD) category is assigned
    the vulnerable (VU) probability,
    see Bland et al. (2015) for explanation.
    
    Args:
        cat (str): IUCN category
            - DD = Data Deficient
            - LC = Least Concern
            - NT = Near Threatened
            - VU = Vulnerable
            - EN = Endangered
            - CR = Critically Endangered
            - EW = Extinct in the wild 
            - EX = Extinct
        
    Returns:
        float: probability of extinction
        
    References:
        Bland et al. (2015) "Predicting the conservation status of data-deficient species" 
            https://doi.org/10.1111/cobi.12372
        Davis et al. (2018) "Mammal diversity will take millions of years to recover from the current biodiversity crisis"
            https://doi.org/10.1073/pnas.1804906115
        Farooq et al. (2020) "WEGE: A new metric for ranking locations for biodiversity conservation" 
            https://doi.org/10.1111/ddi.13148
    '''
    cat_to_risk = dict(
        DD=0.0513, # using Bland et al. (2015) assumption
        LC=0.0009,
        NT=0.0071,
        VU=0.0513,
        EN=0.4276,
        CR=0.9688,
        EW=1.0,
        EX=1.0
    )

    if cat_to_risk.get(cat) is None:
        raise ValueError("Invalid value for 'cat', expected one of 'DD', 'LC', 'NT', 'VU', 'EN', 'CR', EW', 'EX'")
    
    return cat_to_risk.get(cat)

In [2]:
def weighted_endemism(species_area: float=None, total_area: float=None, power: float=None) -> float:
    '''calculates the endemism score for a single species
    following Crisp et al. (2002) and using the scaling power of 0.5
    from Farooq et al. (2020).
    
    `("area species is found" / "total area")^(power)`
    
    Args:
        species_area (float): amount of area within the total where species is found
        total_area (float): total area where searching for species
        power(float): raise each endemism value to this power[default: 0.5]
        
    Returns:
        float: endemism score for the species
    
    References:
        Crisp et al. (2002) "Endemism in the Australian flora"
            https://onlinelibrary.wiley.com/doi/abs/10.1046/j.1365-2699.2001.00524.x
        Farooq et al. (2020) "WEGE: A new metric for ranking locations for biodiversity conservation" 
            https://doi.org/10.1111/ddi.13148
    '''
    power = 0.5 if power is None else power # see Farooq et al. (2020)
    
    if not isinstance(species_area, (int, float)):
        raise TypeError("species_area must be a number")
        
    if species_area < 0:
        raise ValueError("species_area must be positive")

    if not isinstance(total_area, (int, float)):
        raise TypeError("total_area must be a number")
        
    if total_area <= 0:
        raise ValueError("total_area must be positive and non-zero")
    
    return (species_area/total_area)**power


calculate WEGE from lists of `species_area`, `total_area`, and IUCN `categories`

```
WEGE = sum(
    [endemism(sp_area, tot_area) * extinction_risk(cat) 
     for sp_area, tot_area, cat 
     in zip(species_area, total_area, category)]
)
```

# 1. Read data

In [3]:
import geopandas as gpd

## 1.1 IUCN Redlist
**Note:** this takes a couple minutes to load. Takes longer, and requires AWS account, if loading directly from S3 bucket.

In [11]:
%%time
df_redlist = gpd.read_file('s3://ocean-program/data/processed/ACMC_IUCN_RedList/')

CPU times: user 3min 32s, sys: 8.56 s, total: 3min 41s
Wall time: 6min 17s


In [13]:
df_redlist.head()

,index,ASSESSMENT,ID_NO,BINOMIAL,PRESENCE,ORIGIN,SEASONAL,COMPILER,YEAR,CITATION,LEGEND,SUBSPECIES,SUBPOP,DIST_COMM,ISLAND,TAX_COMM,redlistCat,scientific,geometry
0,3,1506081,177969.0,Acanthurus nigricans,1,1,1,Jonnell Sanciangco,2010,International Union for Conservation of Nature...,Extant (resident),NaN,NaN,NaN,NaN,NaN,Least Concern,Acanthurus nigricans,"MULTIPOLYGON (((109.79501 6.13966, 109.89836 6..."
1,10,1512439,177988.0,Naso annulatus,1,1,1,Jonnell Sanciangco,2010,International Union for Conservation of Nature...,Extant (resident),NaN,NaN,NaN,NaN,NaN,Least Concern,Naso annulatus,"MULTIPOLYGON (((109.79501 6.13966, 109.89836 6..."
2,13,1517273,178002.0,Naso hexacanthus,1,1,1,Jonnell Sanciangco,2010,International Union for Conservation of Nature...,Extant (resident),NaN,NaN,NaN,NaN,NaN,Least Concern,Naso hexacanthus,"MULTIPOLYGON (((99.07516 7.94581, 99.07794 7.9..."
3,97,2412930,195754.0,Gymnothorax meleagris,1,1,1,IUCN Marine Biodiversity Unit/GMSA,2018,IUCN Marine Biodiversity Unit/GMSA,Extant (resident),NaN,NaN,NaN,NaN,NaN,Least Concern,Gymnothorax meleagris,"MULTIPOLYGON (((144.23853 -3.82053, 144.24330 ..."
4,102,2423479,195824.0,Uropterygius macrocephalus,1,1,1,IUCN Marine Biodiversity Unit/GMSA,2018,IUCN Marine Biodiversity Unit/GMSA,Extant (resident),NaN,NaN,NaN,NaN,NaN,Least Concern,Uropterygius macrocephalus,"MULTIPOLYGON (((144.23853 -3.82053, 144.24330 ..."


In [14]:
# replaces long RedList name with two-letter code
long_to_short = {
    'Data Deficient':'DD',
    'Least Concern':'LC',
    'Near Threatened':'NT',
    'Vulnerable':'VU',
    'Endangered':'EN',
    'Critically Endangered':'CR',
    'Extinct In The Wild':'EW',
    'Extinct':'EX'
}

df_redlist['redlistCat'] = df_redlist['redlistCat'].replace(long_to_short)

## 1.2 Coco Marine Conservation Area
- The Coco Marine Conservation Area (ACMC)
- the Bicentennial Marine Management Area (AMMB)
- Cocos Island National Park (PNIC)

In [16]:
%%time
df_acmc = gpd.read_file('s3://ocean-program/data/processed/geospatial/')

CPU times: user 37.3 ms, sys: 27.7 ms, total: 65 ms
Wall time: 1.1 s


# 2. Calculate WEGE

In [17]:
# set coord. references system
# https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.set_crs.html
df_redlist = df_redlist.set_crs('epsg:4326', allow_override=True)
df_acmc = df_acmc.set_crs('epsg:4326', allow_override=True)

In [18]:
df_redlist.crs = {"init":"epsg:4326"}
df_acmc.crs = {"init":"epsg:4326"}

/opt/homebrew/Caskroom/miniconda/base/envs/ocean/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [30]:
# area of ACMC
acmc_area = df_acmc.area

/var/folders/g0/rt7kg75d1bq_ptpk0lzn3m5h0000gn/T/ipykernel_60979/390391726.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  acmc_area = df_acmc.area


In [35]:
# create list of weighted endemism to specified power for each species
# hiding warnings for now

import warnings

with warnings.catch_warnings(record=True):
    power = 0.5
    WE_list = []
    for _, row in df_redlist.iterrows():
        df_intersect = df_redlist.loc[df_redlist.BINOMIAL==row.BINOMIAL].intersection(df_acmc)
        species_area_within_acmc = df_intersect.area
        species_redlist_area = df_redlist[df_redlist.BINOMIAL==row.BINOMIAL].area
        WE_tmp = ((species_area_within_acmc/acmc_area).sum())**power
        WE_list.append(WE_tmp)

In [43]:
# list of extinction probabilities for each species
ER_list = [extinction_risk(cat) for cat in df_redlist['redlistCat']]

In [49]:
# calculate WEGE index
WEGE = sum([WE * ER for WE, ER in zip(WE_list, ER_list)])

0.0008425635868365434